In [8]:
# !git clone https://github.com/TomerRonen34/mixed-resolution-vit.git
# %mv mixed-resolution-vit/* ./
# %rm -rf mixed-resolution-vit

In [1]:
import torch

from tqdm import tqdm
from time import time

from vit import *
from utils import *
from dataloader import *
from decatt import DecattLoss

torch.manual_seed(0)

In [2]:
device_ids = [i for i in range(torch.cuda.device_count())]
print(device_ids)

[0]


# ImageNet1k

In [3]:
image_size = 256
min_patch_size = 16
max_patch_size = 64
quadtree_num_patches = 64
in_channels = 3
mlp_dim = 512
num_classes = 1000
num_layers = 12
dropout = 0.1
batch_size = 256

lr = 1e-3
weight_decay = 1e-3
num_epochs = 30

In [4]:
transform_train = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

transform_test = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

dataset = datasets.ImageNet(root='./imagenet', split='val', transform=transform_train)

idx = int(len(dataset) * 0.75)
trainset, testset = torch.utils.data.random_split(dataset, [idx, len(dataset) - idx])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=1)

In [ ]:
num_heads = 3
embed_dim = 64 * num_heads

model = MixedResViT(image_size, min_patch_size, max_patch_size, quadtree_num_patches, in_channels, 
                    embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout)                                                                                          
model = torch.nn.DataParallel(model.cuda(), device_ids=device_ids).cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
train("vit_mrt_imagenet", model, criterion, optimizer, num_epochs, trainloader, testloader)

 13%|█▎        | 4/30 [49:51<4:48:11, 665.05s/it, Epoch=4, Train Accuracy=0.253, Training Loss=6.79, Validation Accuracy=0.28]   